#Train ru gpt with apex

In [ ]:
import torch
torch.__version__

'1.9.0+cu102'

In [ ]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.8 MB 8.5 MB/s 
     |████████████████████████████████| 2.9 MB 69.2 MB/s 
     |████████████████████████████████| 895 kB 72.0 MB/s 


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

--2021-09-16 09:24:11--  https://tlk.s3.yandex.net/dataset/TlkPersonaChatRus.zip
Resolving tlk.s3.yandex.net (tlk.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to tlk.s3.yandex.net (tlk.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6139432 (5.9M) [application/zip]
Saving to: ‘TlkPersonaChatRus.zip’

TlkPersonaChatRus.z 100%[===================>]   5.85M  3.97MB/s    in 1.5s    

2021-09-16 09:24:13 (3.97 MB/s) - ‘TlkPersonaChatRus.zip’ saved [6139432/6139432]

Archive:  TlkPersonaChatRus.zip
   creating: TlkPersonaChatRus/
  inflating: TlkPersonaChatRus/dialogues.tsv  
  inflating: TlkPersonaChatRus/readme_TlkPersonaChatRus.txt  
  inflating: TlkPersonaChatRus/profiles.tsv  


In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

# download simple training set

In [ ]:
!wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
!wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --fp16_opt_level 'O4' \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

# Simple inference

In [ ]:
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

np.random.seed(42)
torch.manual_seed(42)

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/essays")
model = GPT2LMHeadModel.from_pretrained("models/essays").cuda()

text = "<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: "
inpt = tok.encode(text, return_tensors="pt")
out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)
print(tok.decode(out[0]))

# Also simple api

In [ ]:
from fastapi import FastAPI
app = FastAPI()
from fastapi import FastAPI

from fastapi.middleware.cors import CORSMiddleware
from starlette.middleware import Middleware
from starlette.middleware.cors import CORSMiddleware


import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device='cuda'
BACKBONE = "content/weights_push_v_may"

tok = GPT2Tokenizer.from_pretrained(BACKBONE)
model = GPT2LMHeadModel.from_pretrained(BACKBONE).to(device)
#model = model.half()

def generate(text,lenght=100,temp=1,num=2):
  
  

  inpt = tok.encode(text, return_tensors="pt").to(device)

  max_length=lenght

  out = model.generate(inpt,  
          do_sample=True,
		       top_k=1000,
		       top_p=0.95,
		       length=lenght,		       	 
           temperature=temp)
  decoded = tok.decode(out[0])
  return decoded



origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
@app.get("/")
def main():
    return "go to /push/?message=пидр"



@app.get('/push/')
def detect_spam_query(text: str,temp: float):
  
  return {"text":generate(text,temp=temp)}

import nest_asyncio
#from pyngrok import ngrok
import uvicorn
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=5000,timeout_keep_alive=10000)
